In [3]:
#!/usr/bin/env python3
import os
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# ============================================================
# LIVE TRADE GENERATOR (OPTION A)
# ============================================================
# PURPOSE
# - Generate "planned trades" off Wednesday close (signal day)
# - WITHOUT requiring Thursday open data
# - WITHOUT mutating the authoritative live portfolio file
#
# KEY RULES
# 1) live_portfolio.csv is AUTHORITATIVE and should ONLY be updated by reconciliation (executed fills)
# 2) This script writes PLANNED outputs only:
#    - weekly_trades_signal_YYYYMMDD.csv
#    - planned_portfolio_signal_YYYYMMDD.csv
#    - master_trades.csv (planned trades ledger; de-duped)
#    - weekly_rankings_signal_YYYYMMDD.csv (NEW)
#    - master_rankings.csv (NEW)
#
# OPTION A EXECUTION PRICING
# - We do NOT know Thursday open on Wednesday night.
# - So we estimate execution price = Wednesday close_adj (signal-day close).
# ============================================================

# ============================================================
# CONFIG (from your live settings)
# ============================================================

UNIVERSE_FILE   = "./12-tradable_sp500_universe/12-tradable_sp500_universe.parquet"
ATR20_DIR       = "./4-ATR20_adjusted_All_Prices"
SPY_FILE        = "./8-SPY_200DMA_market_regime/8-SPY_200DMA_regime.parquet"

# LIVE OUTPUT ROOT
LIVE_ROOT = "./27a-2G_live_trading"
MASTER_TRADES_FILE        = f"{LIVE_ROOT}/master_trades.csv"
MASTER_RANKINGS_FILE      = f"{LIVE_ROOT}/master_rankings.csv"
WEEKLY_TRADES_DIR         = f"{LIVE_ROOT}/weekly_trades"
WEEKLY_RANKINGS_DIR       = f"{LIVE_ROOT}/weekly_rankings"
WEEKLY_PORT_DIR           = f"{LIVE_ROOT}/weekly_portfolios"
LIVE_PORTFOLIO_FILE       = f"{LIVE_ROOT}/live_portfolio.csv"              # AUTHORITATIVE (DO NOT WRITE)
PLANNED_LIVE_PORT_FILE    = f"{LIVE_ROOT}/planned_live_portfolio.csv"       # PLANNING STATE (OK TO WRITE)

for d in [LIVE_ROOT, WEEKLY_TRADES_DIR, WEEKLY_RANKINGS_DIR, WEEKLY_PORT_DIR]:
    os.makedirs(d, exist_ok=True)

# Start from this signal date onward
START_TRADING         = pd.Timestamp("2025-12-17")

INITIAL_CAPITAL       = 345000.0
TOP_PERCENTILE        = 0.95
REBALANCE_DAY         = "Wednesday"
TRADING_DAYS_PER_YEAR = 252

MIN_CASH_RESERVE = 20000.0

# --- SPY REGIME CONFIRMATION PERIOD (MATCH BACKTEST ENGINE) ---
# Number of consecutive days SPY must stay above/below 200 DMA before confirming regime change
# Set to 1 for original behavior (immediate flip on crossover)
# Set to 5, 10, etc. to filter out whipsaw signals around the 200 DMA
SPY_REGIME_CONFIRM_DAYS = 1

# --- Position cap (MATCH BACKTEST ENGINE) ---
MAX_POSITION_WEIGHT = 0.12   # 12% max position weight at exec-proxy (Wednesday close)

# Turnover / trade filters (kept consistent with your engine)
DRIFT_THRESHOLD          = 0.05
MIN_TRADE_VALUE          = 10000.0
MIN_NEW_POSITION_WEIGHT  = 0.005

# ============================================================
# EXECUTION PRICING MODE (Option A)
# ============================================================
EXECUTION_PRICE_MODE = "WED_CLOSE_AS_THU_OPEN_ESTIMATE"

print("=== LIVE TRADE GENERATOR (BACKTEST-CONGRUENT, OPTION A) ===")
print(f"Execution pricing: {EXECUTION_PRICE_MODE}")
print(f"Start trading:     {START_TRADING.date()}")
print(f"Rebalance day:     {REBALANCE_DAY}")
print(f"Max position cap:  {MAX_POSITION_WEIGHT:.0%}")
print(f"SPY Regime Confirmation Period: {SPY_REGIME_CONFIRM_DAYS} day(s)")
print("===============================================================\n")

# ============================================================
# EXECUTION DIAGNOSTICS (kept; lightweight)
# ============================================================

exec_diag = {
    "weeks_seen": 0,
    "weeks_written": 0,
    "orders_generated": 0,
    "dropped_missing_price": 0,
    "dropped_min_trade": 0,
    "dropped_cash_floor_buy": 0,
    "dropped_spy_regime_buy": 0,
    "dropped_min_new_weight": 0,
    "dropped_deduped_existing": 0,
}

# ============================================================
# SPY REGIME CONFIRMATION FUNCTION (MATCH BACKTEST ENGINE)
# ============================================================

def create_confirmed_regime(raw_regime: np.ndarray, confirm_days: int) -> np.ndarray:
    """
    Create a confirmed regime signal that requires N consecutive days
    above/below the 200 DMA before flipping the regime.
    
    Parameters:
    -----------
    raw_regime : array of 0/1 (0 = below 200 DMA, 1 = above 200 DMA)
    confirm_days : number of consecutive days required to confirm regime change
    
    Returns:
    --------
    confirmed_regime : array of 0/1 with smoothed regime signal
    """
    if confirm_days <= 1:
        return raw_regime.copy()
    
    n = len(raw_regime)
    confirmed = np.zeros(n, dtype=int)
    
    # Start with the initial regime (use first value)
    current_regime = raw_regime[0]
    consecutive_count = 0
    
    for i in range(n):
        if raw_regime[i] == current_regime:
            # Same as current confirmed regime
            consecutive_count = 0  # Reset counter for opposite regime
            confirmed[i] = current_regime
        else:
            # Different from current confirmed regime
            consecutive_count += 1
            
            if consecutive_count >= confirm_days:
                # Confirm the regime change
                current_regime = raw_regime[i]
                consecutive_count = 0
            
            confirmed[i] = current_regime
    
    return confirmed

# ============================================================
# FAST PRICE LOOKUP (close fallback)
# ============================================================

def fast_price_lookup(px_array, date_val):
    date_val = np.datetime64(date_val, "ns")
    dates = px_array["date"]
    idx = np.searchsorted(dates, date_val, side="right") - 1
    if idx < 0:
        return np.nan
    return px_array["px"][idx]

# ============================================================
# SNAPSHOT PORTFOLIO (close + "exec proxy")
# ============================================================

def snapshot_portfolio_close(date, cash, positions, px_by_ticker_close):
    """
    Snapshot using close prices (fallback).
    NOTE: In Option A we primarily size using exec proxy prices from signal-day close.
    """
    equity = 0.0
    for t, pos in positions.items():
        if t not in px_by_ticker_close:
            continue
        px = fast_price_lookup(px_by_ticker_close[t], date)
        if not np.isnan(px):
            equity += int(pos["shares"]) * float(px)
    return equity, cash + equity, len(positions)

def snapshot_portfolio_exec_proxy(exec_date, cash, positions, exec_px_map, px_by_ticker_close_fallback):
    """
    Snapshot using "execution proxy" prices.
    Option A: exec_px_map is built from signal-day close_adj (Wednesday close).
    """
    equity = 0.0
    for t, pos in positions.items():
        px = exec_px_map.get(t, np.nan)
        if pd.isna(px) or px <= 0:
            if t in px_by_ticker_close_fallback:
                px = fast_price_lookup(px_by_ticker_close_fallback[t], exec_date)
        if pd.notna(px) and px > 0:
            equity += int(pos["shares"]) * float(px)
    return equity, cash + equity, len(positions)

# ============================================================
# LIVE PORTFOLIO LOAD / SAVE
# ============================================================

def load_live_portfolio(path: str):
    """
    Loads AUTHORITATIVE live portfolio state (from executed trades).
    This file should NOT be mutated by this generator.
    """
    if not os.path.exists(path):
        return float(INITIAL_CAPITAL), {}  # cash, positions

    p = pd.read_csv(path)
    if p.empty:
        return float(INITIAL_CAPITAL), {}

    cash = float(p["cash"].iloc[0]) if "cash" in p.columns else float(INITIAL_CAPITAL)

    positions = {}
    if "ticker" in p.columns and "shares" in p.columns:
        for _, r in p.iterrows():
            t = str(r["ticker"]).strip()
            if not t or t == "nan":
                continue
            sh = int(r["shares"])
            if sh <= 0:
                continue
            entry = float(r["entry_price"]) if "entry_price" in p.columns and pd.notna(r["entry_price"]) else np.nan
            positions[t] = {"shares": sh, "entry": entry}

    return cash, positions

def save_portfolio_snapshot(path: str, asof_date: pd.Timestamp, cash: float, positions: dict):
    """
    Writes a portfolio snapshot (planning or reporting).
    Safe to use for planned portfolios and weekly snapshots.
    """
    rows = []
    if positions:
        for t, pos in sorted(positions.items()):
            rows.append({
                "asof_date": asof_date.date().isoformat(),
                "cash": float(cash),
                "ticker": t,
                "shares": int(pos["shares"]),
                "entry_price": float(pos.get("entry", np.nan)) if pd.notna(pos.get("entry", np.nan)) else np.nan,
            })
    else:
        rows.append({
            "asof_date": asof_date.date().isoformat(),
            "cash": float(cash),
            "ticker": "",
            "shares": 0,
            "entry_price": np.nan,
        })

    pd.DataFrame(rows).to_csv(path, index=False)

# ============================================================
# HELPERS
# ============================================================

def is_rebalance_day(date: pd.Timestamp) -> bool:
    return date.day_name() == REBALANCE_DAY

def get_signal_close(day_df, ticker):
    row = day_df.loc[day_df["ticker"] == ticker, "close_adj"]
    if row.empty:
        return np.nan
    return float(row.iloc[0])

def make_plan_id(signal_date: pd.Timestamp, exec_date: pd.Timestamp, ticker: str, side: str) -> str:
    """
    Deterministic ID so repeated runs don't duplicate planned trades in master_trades.csv
    """
    return f"PLAN-{signal_date.strftime('%Y%m%d')}-{exec_date.strftime('%Y%m%d')}-{ticker}-{side}"

def categorize_no_trade(row):
    """Determine why a stock didn't trade"""
    
    # Already at target (within drift threshold)
    if abs(row['weight_change']) < DRIFT_THRESHOLD:
        return 'Within drift threshold'
    
    # New position but too small
    if row['current_shares'] == 0 and row['target_weight'] < MIN_NEW_POSITION_WEIGHT:
        return 'New position too small'
    
    # Trade value too small
    if abs(row['shares_change'] * row['close_adj']) < MIN_TRADE_VALUE:
        return 'Trade value too small'
    
    # Would buy but SPY regime prevents it
    if row['shares_change'] > 0 and not row['spy_above_200dma']:
        return 'SPY regime prevented buy'
    
    # Insufficient cash
    if row['shares_change'] > 0:
        return 'Insufficient cash'
    
    return 'Other'

def cap_and_redistribute_weights(w: np.ndarray, cap: float) -> np.ndarray:
    """
    MATCH BACKTEST ENGINE:
    Caps weights at `cap` and redistributes any excess proportionally
    to the remaining (uncapped) names.

    If not feasible to fully invest under cap (N*cap < 1), it will cap
    and leave leftover unallocated (cash drag).
    """
    w = np.asarray(w, dtype=float).copy()
    if w.size == 0:
        return w

    s = w.sum()
    if s > 0:
        w /= s

    # Not feasible to be fully invested under cap
    if w.size * cap < 1.0:
        return np.minimum(w, cap)

    # Iteratively cap and redistribute
    for _ in range(10_000):
        over = w > cap
        if not over.any():
            break
        excess = (w[over] - cap).sum()
        w[over] = cap
        under = ~over
        under_sum = w[under].sum()
        if under_sum <= 0:
            break
        w[under] += excess * (w[under] / under_sum)

    return w

# ============================================================
# TRADING CALENDAR HELPER
# ============================================================

def get_next_trading_day(signal_date: pd.Timestamp, trading_calendar: list) -> pd.Timestamp:
    """
    Returns the next actual trading day after signal_date.
    Uses the trading calendar (from universe or SPY data) to skip weekends and holidays.
    
    If signal_date is beyond the calendar, falls back to finding the next weekday
    that isn't a known US market holiday.
    """
    signal_date = pd.Timestamp(signal_date).normalize()
    
    # First, try to find it in the trading calendar
    for d in trading_calendar:
        if d > signal_date:
            return d
    
    # Fallback: iterate forward until we find a likely trading day
    # (Skip weekends and known US market holidays)
    US_MARKET_HOLIDAYS = {
        # 2025
        (2025, 1, 1),   # New Year's Day
        (2025, 1, 20),  # MLK Day
        (2025, 2, 17),  # Presidents Day
        (2025, 4, 18),  # Good Friday
        (2025, 5, 26),  # Memorial Day
        (2025, 6, 19),  # Juneteenth
        (2025, 7, 4),   # Independence Day
        (2025, 9, 1),   # Labor Day
        (2025, 11, 27), # Thanksgiving
        (2025, 12, 25), # Christmas
        # 2026
        (2026, 1, 1),   # New Year's Day
        (2026, 1, 19),  # MLK Day
        (2026, 2, 16),  # Presidents Day
        (2026, 4, 3),   # Good Friday
        (2026, 5, 25),  # Memorial Day
        (2026, 6, 19),  # Juneteenth
        (2026, 7, 3),   # Independence Day (observed)
        (2026, 9, 7),   # Labor Day
        (2026, 11, 26), # Thanksgiving
        (2026, 12, 25), # Christmas
    }
    
    candidate = signal_date + pd.Timedelta(days=1)
    for _ in range(10):  # Max 10 days forward (handles long weekends)
        # Skip weekends
        if candidate.weekday() >= 5:  # Saturday=5, Sunday=6
            candidate += pd.Timedelta(days=1)
            continue
        # Skip known holidays
        if (candidate.year, candidate.month, candidate.day) in US_MARKET_HOLIDAYS:
            candidate += pd.Timedelta(days=1)
            continue
        # Found a likely trading day
        return candidate
    
    # Ultimate fallback (shouldn't reach here)
    return signal_date + pd.Timedelta(days=1)

# ============================================================
# LOAD UNIVERSE
# ============================================================

df = pd.read_parquet(UNIVERSE_FILE)
df["date"] = pd.to_datetime(df["date"])

df["slope_adj"] = pd.to_numeric(df["slope_adj"], errors="coerce")
df["close_adj"] = pd.to_numeric(df["close_adj"], errors="coerce")
df["open_adj"]  = pd.to_numeric(df.get("open_adj", np.nan), errors="coerce")  # may exist; not used in Option A

# Some universe files use in_sp500; if missing treat as True
if "in_sp500" not in df.columns:
    df["in_sp500"] = True

print(f"Loaded universe: {len(df):,} rows")

# ============================================================
# MERGE ATR20 PER-TICKER
# ============================================================

atr20_map = {}
for f in os.listdir(ATR20_DIR):
    if not f.endswith(".parquet"):
        continue
    t = f.replace(".parquet", "")
    tmp = pd.read_parquet(os.path.join(ATR20_DIR, f))
    if "atr20" not in tmp.columns:
        continue
    tmp["date"] = pd.to_datetime(tmp["date"])
    atr20_map[t] = tmp[["date", "atr20"]]

rows = []
for t, sub in df.groupby("ticker", sort=False):
    if t in atr20_map:
        rows.append(sub.merge(atr20_map[t], on="date", how="left"))
    else:
        sub = sub.copy()
        sub["atr20"] = np.nan
        rows.append(sub)

df = pd.concat(rows, ignore_index=True)
print(f"Universe with ATR20 merged: {len(df):,} rows")

# ============================================================
# LOAD SPY REGIME (WITH CONFIRMATION FILTER)
# ============================================================

spy = pd.read_parquet(SPY_FILE)

if spy.index.name in ["Date", "date", None]:
    spy = spy.reset_index().rename(columns={"index": "date", "Date": "date"})
spy["date"] = pd.to_datetime(spy["date"])

if "market_regime" not in spy.columns:
    raise ValueError("SPY file missing 'market_regime' column")

# Get raw regime signal (1 = above 200 DMA, 0 = below)
raw_regime = spy["market_regime"].astype(int).values

# Apply confirmation period filter (MATCH BACKTEST ENGINE)
confirmed_regime = create_confirmed_regime(raw_regime, SPY_REGIME_CONFIRM_DAYS)
spy["spy_above_200dma"] = confirmed_regime == 1

# Report regime statistics
n_bull_raw = (raw_regime == 1).sum()
n_bear_raw = (raw_regime == 0).sum()
n_bull_confirmed = (confirmed_regime == 1).sum()
n_bear_confirmed = (confirmed_regime == 0).sum()

print(f"\nSPY Regime Statistics:")
print(f"  Raw regime:       {n_bull_raw:,} bull days, {n_bear_raw:,} bear days")
print(f"  Confirmed regime: {n_bull_confirmed:,} bull days, {n_bear_confirmed:,} bear days")
if SPY_REGIME_CONFIRM_DAYS > 1:
    regime_changes_raw = np.sum(np.diff(raw_regime) != 0)
    regime_changes_confirmed = np.sum(np.diff(confirmed_regime) != 0)
    print(f"  Regime changes:   {regime_changes_raw} raw → {regime_changes_confirmed} confirmed")

spy_regime_map = spy.set_index("date")["spy_above_200dma"].to_dict()

# ============================================================
# PREP GROUPED DATA + DATE MAPS
# ============================================================

df_by_date = {d: sub for d, sub in df.groupby("date")}

# close price history (fallback)
px_by_ticker_close = {}
for t, sub in df.groupby("ticker", sort=False):
    sub = sub.sort_values("date")
    arr = np.zeros(len(sub), dtype=[("date", "datetime64[ns]"), ("px", "float64")])
    arr["date"] = sub["date"].values.astype("datetime64[ns]")
    arr["px"]   = sub["close_adj"].astype(float).values
    px_by_ticker_close[t] = arr

dates = sorted(df_by_date.keys())

# Build trading calendar for next-day lookups (combine universe dates + SPY dates for completeness)
trading_calendar = sorted(set(dates) | set(spy["date"].dropna()))

# Next-trading-day map from available universe dates
next_date_map = {d: dates[i + 1] if i + 1 < len(dates) else None for i, d in enumerate(dates)}

# Next-trading-day map from available universe dates
next_date_map = {d: dates[i + 1] if i + 1 < len(dates) else None for i, d in enumerate(dates)}

# ============================================================
# LOAD AUTHORITATIVE LIVE PORTFOLIO (DO NOT WRITE IT HERE)
# ============================================================

cash_real, positions_real = load_live_portfolio(LIVE_PORTFOLIO_FILE)

# PLANNING STATE (we mutate this in-sim to build weekly plans)
cash_plan = float(cash_real)
positions_plan = {k: v.copy() for k, v in positions_real.items()}

print(f"\nLoaded AUTHORITATIVE live portfolio state (from {LIVE_PORTFOLIO_FILE}):")
print(f"  Cash:      {cash_real:,.2f}")
print(f"  Positions: {len(positions_real)}\n")

# ============================================================
# BUILD SIGNAL DATES (DO NOT REQUIRE NEXT DAY DATA EXISTENCE)
# ============================================================

signal_dates = [
    d for d in dates
    if d >= START_TRADING
    and is_rebalance_day(d)
]

if not signal_dates:
    raise RuntimeError("No eligible rebalance Wednesdays found after START_TRADING.")

print(f"Eligible rebalance weeks: {len(signal_dates)}\n")

# ============================================================
# MASTER TRADES & RANKINGS LOAD (append mode + DE-DUP)
# ============================================================

def load_master_trades(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame()
    try:
        return pd.read_csv(path)
    except Exception:
        return pd.DataFrame()

master_df = load_master_trades(MASTER_TRADES_FILE)

# Ensure master has plan_id column if present from older runs
if not master_df.empty and "plan_id" not in master_df.columns:
    master_df["plan_id"] = np.nan

# Load master rankings (NEW)
master_rankings = load_master_trades(MASTER_RANKINGS_FILE)

# ============================================================
# MAIN LOOP: GENERATE TRADES FOR EVERY SIGNAL WEEK (PLANNING)
# ============================================================

for signal_date in signal_dates:
    exec_diag["weeks_seen"] += 1

    # --------------------------------------------------------
    # Exec date is "next trading day" if available, else calendar +1 day.
    # This is only a LABEL for the planned execution day.
    # Option A pricing uses signal-day close regardless.
    # --------------------------------------------------------
    trade_date = next_date_map.get(signal_date)
    if trade_date is None:
        # Use trading calendar to find actual next trading day (handles holidays)
        trade_date = get_next_trading_day(signal_date, trading_calendar)

    # Pull signal-day universe slice
    if signal_date not in df_by_date:
        continue

    day = df_by_date[signal_date]
    if day is None or day.empty:
        continue

    # -------------------------
    # SPY regime for this week (signal-day close, with confirmation filter applied)
    # -------------------------
    spy_above_200 = bool(spy_regime_map.get(signal_date, True))
    can_buy_next_open = spy_above_200

    # -------------------------
    # Rank / top group
    # -------------------------
    rankable = day[
        (day["slope_adj"].notna()) &
        (day["in_sp500"] == True)
    ].copy()

    if rankable.empty:
        continue

    rankable = rankable.sort_values("slope_adj", ascending=False)
    cutoff = rankable["slope_adj"].quantile(TOP_PERCENTILE)
    top_group = rankable[rankable["slope_adj"] >= cutoff].copy()

    if top_group.empty:
        continue

    top_group = top_group.sort_values("slope_adj", ascending=False)
    top_group["slope_rank_within_top"] = np.arange(1, len(top_group) + 1)
    rank_map = dict(zip(top_group["ticker"], top_group["slope_rank_within_top"]))
    top_tickers = set(top_group["ticker"].values)

    # -------------------------
    # Exec price map (Option A proxy) = signal-day close_adj
    # -------------------------
    exec_px_map = day.set_index("ticker")["close_adj"].to_dict()

    # =========================================================
    # WEEKLY TRADE GENERATION (MUTATES PLANNING STATE ONLY)
    # =========================================================
    weekly_rows = []

    # 1) EXIT SELLS FIRST
    exit_tickers = [t for t in list(positions_plan.keys()) if t not in top_tickers]

    for t in exit_tickers:
        pos_shares = int(positions_plan[t]["shares"])

        px = exec_px_map.get(t, np.nan)
        if pd.isna(px) or px <= 0:
            if t in px_by_ticker_close:
                px = fast_price_lookup(px_by_ticker_close[t], signal_date)

        if pd.isna(px) or px <= 0:
            exec_diag["dropped_missing_price"] += 1
            continue

        px = float(px)
        est_value = pos_shares * px

        cash_before = cash_plan
        cash_plan += est_value

        del positions_plan[t]

        _, port_after, npos_after = snapshot_portfolio_exec_proxy(
            trade_date, cash_plan, positions_plan, exec_px_map, px_by_ticker_close
        )

        side = "SELL"
        weekly_rows.append({
            "plan_id": make_plan_id(signal_date, trade_date, t, side),
            "created_ts": datetime.now().isoformat(timespec="seconds"),
            "signal_date": signal_date.date().isoformat(),
            "exec_date": trade_date.date().isoformat(),
            "ticker": t,
            "side": side,
            "shares": pos_shares,
            "est_exec_px": px,
            "est_value": float(est_value),
            "reason": "not_in_top_quintile",
            "slope_rank": int(rank_map.get(t, 9999)),
            "spy_above_200dma": spy_above_200,
            "cash_before": float(cash_before),
            "cash_after": float(cash_plan),
            "portfolio_after": float(port_after),
            "num_positions_after": int(npos_after),
        })

    # 2) REVALUE PORTFOLIO AT "EXEC TIME" (proxy) AFTER EXITS
    equity_exec, portfolio_exec, _ = snapshot_portfolio_exec_proxy(
        trade_date, cash_plan, positions_plan, exec_px_map, px_by_ticker_close
    )
    effective_equity = max(portfolio_exec - MIN_CASH_RESERVE, 0.0)

    # 3) BUILD TARGETS (signal-day atr/slope) AND SIZE USING exec_px_map (proxy)
    tg = top_group.copy()
    tg = tg[
        tg["atr20"].notna() &
        (tg["atr20"] > 0) &
        tg["close_adj"].notna() &
        (tg["close_adj"] > 0)
    ].copy()

    # =========================================================
    # NEW: STORE PRE-FILTER RANKINGS
    # =========================================================
    weekly_rankings = []

    if not tg.empty:
        inv_vol = 1.0 / tg["atr20"].astype(float)
        total_inv_vol = inv_vol.sum()

        if total_inv_vol > 0:
            tg["inv_vol"] = inv_vol

            # ---- MATCH BACKTEST ENGINE: compute raw weights then cap+redistribute ----
            tg["raw_weight"] = tg["inv_vol"] / total_inv_vol
            w_cap = cap_and_redistribute_weights(tg["raw_weight"].to_numpy(), MAX_POSITION_WEIGHT)
            tg["weight"] = w_cap

            # Size off effective equity (after cash reserve) using capped weights
            tg["target_value"] = effective_equity * tg["weight"]

            # Attach proxy execution price (signal-day close)
            tg = tg.merge(
                pd.Series(exec_px_map, name="exec_px_est"),
                left_on="ticker",
                right_index=True,
                how="inner"
            )
            tg = tg[tg["exec_px_est"].notna() & (tg["exec_px_est"] > 0)].copy()

            # ---- Hard safety: no target value over 12% of proxy portfolio value ----
            tg["target_value"] = np.minimum(tg["target_value"], MAX_POSITION_WEIGHT * portfolio_exec)

            # target_shares uses proxy exec price (signal-day close)
            tg["target_shares"] = np.floor(tg["target_value"] / tg["exec_px_est"]).astype(int)
            tg = tg[tg["target_shares"] > 0].copy()

            total_portfolio_value_exec = float(portfolio_exec)

            # deterministic order (match backtest: slope descending)
            tg = tg.sort_values("slope_adj", ascending=False)

            # -------------------------
            # CAPTURE ALL RANKED STOCKS BEFORE FILTERING
            # -------------------------
            for _, row in tg.iterrows():
                t = str(row["ticker"])
                rank = int(rank_map.get(t, 9999))
                px = float(row["exec_px_est"])
                target_sh = int(row["target_shares"])
                current_sh = int(positions_plan.get(t, {}).get("shares", 0))
                
                target_value = target_sh * px
                target_weight = (target_value / total_portfolio_value_exec) if total_portfolio_value_exec > 0 else 0.0
                
                current_value = current_sh * px
                current_weight = (current_value / total_portfolio_value_exec) if total_portfolio_value_exec > 0 else 0.0
                
                weekly_rankings.append({
                    "signal_date": signal_date.date().isoformat(),
                    "exec_date": trade_date.date().isoformat(),
                    "ticker": t,
                    "slope_rank": rank,
                    "slope_adj": float(row["slope_adj"]),
                    "atr20": float(row["atr20"]),
                    "close_adj": float(row["close_adj"]),
                    "raw_weight": float(row["raw_weight"]),
                    "capped_weight": float(row["weight"]),
                    "target_value": target_value,
                    "target_weight": target_weight,
                    "target_shares": target_sh,
                    "current_shares": current_sh,
                    "current_value": current_value,
                    "current_weight": current_weight,
                    "weight_change": target_weight - current_weight,
                    "shares_change": target_sh - current_sh,
                    "spy_above_200dma": spy_above_200,
                    "portfolio_value": total_portfolio_value_exec,
                })

            # -------------------------
            # NOW GENERATE ACTUAL TRADES (WITH FILTERS)
            # -------------------------
            for _, row in tg.iterrows():
                t = str(row["ticker"])
                px = float(row["exec_px_est"])
                if not (px > 0):
                    exec_diag["dropped_missing_price"] += 1
                    continue

                target_sh = int(row["target_shares"])
                current_sh = int(positions_plan.get(t, {}).get("shares", 0))

                # ---- MATCH BACKTEST ENGINE: share-level cap enforcement ----
                max_shares_allowed = (
                    int(np.floor((MAX_POSITION_WEIGHT * total_portfolio_value_exec) / px))
                    if total_portfolio_value_exec > 0 else 0
                )
                target_sh = min(target_sh, max_shares_allowed)

                target_value = target_sh * px
                target_weight = (target_value / total_portfolio_value_exec) if total_portfolio_value_exec > 0 else 0.0

                current_value = current_sh * px
                current_weight = (current_value / total_portfolio_value_exec) if total_portfolio_value_exec > 0 else 0.0

                weight_diff = abs(target_weight - current_weight)
                is_new_position = (current_sh == 0)

                # ---- MATCH BACKTEST ENGINE: force trim if currently breaching cap ----
                cap_breach = (current_weight > MAX_POSITION_WEIGHT + 1e-9)

                # Skip micro rebalances unless we need to fix a cap breach
                if (weight_diff < DRIFT_THRESHOLD) and (not cap_breach):
                    continue

                # ---------------- SELL (rebalance down / cap trim) ----------------
                if target_sh < current_sh:
                    trade_shares = current_sh - target_sh
                    est_value = trade_shares * px

                    if est_value < MIN_TRADE_VALUE:
                        exec_diag["dropped_min_trade"] += 1
                        continue

                    cash_before = cash_plan
                    cash_plan += est_value

                    new_sh = current_sh - trade_shares
                    if new_sh <= 0:
                        positions_plan.pop(t, None)
                    else:
                        positions_plan[t]["shares"] = int(new_sh)

                    _, port_after, npos_after = snapshot_portfolio_exec_proxy(
                        trade_date, cash_plan, positions_plan, exec_px_map, px_by_ticker_close
                    )

                    side = "SELL"
                    weekly_rows.append({
                        "plan_id": make_plan_id(signal_date, trade_date, t, side),
                        "created_ts": datetime.now().isoformat(timespec="seconds"),
                        "signal_date": signal_date.date().isoformat(),
                        "exec_date": trade_date.date().isoformat(),
                        "ticker": t,
                        "side": side,
                        "shares": int(trade_shares),
                        "est_exec_px": px,
                        "est_value": float(est_value),
                        "reason": "rebalance_down" if not cap_breach else "cap_trim",
                        "slope_rank": int(rank_map.get(t, 9999)),
                        "spy_above_200dma": spy_above_200,
                        "cash_before": float(cash_before),
                        "cash_after": float(cash_plan),
                        "portfolio_after": float(port_after),
                        "num_positions_after": int(npos_after),
                    })

                # ---------------- BUY (rebalance up / new entry) ----------------
                elif target_sh > current_sh:
                    if not can_buy_next_open:
                        exec_diag["dropped_spy_regime_buy"] += 1
                        continue

                    trade_shares = target_sh - current_sh
                    est_value = trade_shares * px

                    if is_new_position and target_weight < MIN_NEW_POSITION_WEIGHT:
                        exec_diag["dropped_min_new_weight"] += 1
                        continue

                    if est_value < MIN_TRADE_VALUE:
                        exec_diag["dropped_min_trade"] += 1
                        continue

                    if est_value > cash_plan - MIN_CASH_RESERVE:
                        exec_diag["dropped_cash_floor_buy"] += 1
                        continue

                    cash_before = cash_plan
                    cash_plan -= est_value

                    if t in positions_plan:
                        positions_plan[t]["shares"] = int(current_sh + trade_shares)
                    else:
                        positions_plan[t] = {"shares": int(trade_shares), "entry": px}

                    _, port_after, npos_after = snapshot_portfolio_exec_proxy(
                        trade_date, cash_plan, positions_plan, exec_px_map, px_by_ticker_close
                    )

                    side = "BUY"
                    weekly_rows.append({
                        "plan_id": make_plan_id(signal_date, trade_date, t, side),
                        "created_ts": datetime.now().isoformat(timespec="seconds"),
                        "signal_date": signal_date.date().isoformat(),
                        "exec_date": trade_date.date().isoformat(),
                        "ticker": t,
                        "side": side,
                        "shares": int(trade_shares),
                        "est_exec_px": px,
                        "est_value": float(est_value),
                        "reason": "rebalance_up" if current_sh > 0 else "new_entry",
                        "slope_rank": int(rank_map.get(t, 9999)),
                        "spy_above_200dma": spy_above_200,
                        "cash_before": float(cash_before),
                        "cash_after": float(cash_plan),
                        "portfolio_after": float(port_after),
                        "num_positions_after": int(npos_after),
                    })

    # =========================================================
    # MERGE RANKINGS WITH TRADES TO DETERMINE STATUS
    # =========================================================
    
    rankings_df = pd.DataFrame(weekly_rankings)
    trades_df = pd.DataFrame(weekly_rows)
    
    if not rankings_df.empty:
        # Merge to determine which stocks traded
        trades_agg = trades_df.groupby('ticker').agg({
            'side': lambda x: ', '.join(x.unique()),
            'shares': 'sum',
        }).reset_index() if not trades_df.empty else pd.DataFrame(columns=['ticker', 'side', 'shares'])
        
        rankings_df = rankings_df.merge(
            trades_agg,
            on='ticker',
            how='left',
            indicator=True
        )
        
        rankings_df['traded'] = rankings_df['_merge'] == 'both'
        rankings_df['traded_flag'] = rankings_df['traded'].map({True: 'TRADED', False: 'NOT_TRADED'})
        rankings_df = rankings_df.drop('_merge', axis=1)
        
        # Add reasons for not trading
        rankings_df['no_trade_reason'] = ''
        not_traded_mask = ~rankings_df['traded']
        rankings_df.loc[not_traded_mask, 'no_trade_reason'] = rankings_df[not_traded_mask].apply(
            categorize_no_trade, axis=1
        )

    # =========================================================
    # WRITE WEEKLY FILES + APPEND MASTER (DE-DUP) + SAVE PLANNED PORT
    # =========================================================

    weekly_tag = signal_date.strftime("%Y%m%d")
    weekly_trades_file = os.path.join(WEEKLY_TRADES_DIR, f"weekly_trades_signal_{weekly_tag}.csv")
    weekly_rankings_file = os.path.join(WEEKLY_RANKINGS_DIR, f"weekly_rankings_signal_{weekly_tag}.csv")
    weekly_port_file   = os.path.join(WEEKLY_PORT_DIR,   f"weekly_portfolio_after_{weekly_tag}.csv")
    planned_port_file  = os.path.join(WEEKLY_PORT_DIR,   f"planned_portfolio_signal_{weekly_tag}.csv")

    if not trades_df.empty:
        # SELLs first for readability
        trades_df["side_order"] = trades_df["side"].map({"SELL": 0, "BUY": 1}).fillna(9).astype(int)

        trades_df = trades_df.sort_values(
            ["side_order", "slope_rank"],
            ascending=[True, True]
        ).drop(columns=["side_order"])

    # Always write the weekly files (even empty) so you can see "no trades" weeks
    trades_df.to_csv(weekly_trades_file, index=False)
    
    # Write rankings file
    if not rankings_df.empty:
        rankings_df = rankings_df.sort_values("slope_rank")
        rankings_df.to_csv(weekly_rankings_file, index=False)

    # Save planned portfolio snapshots (optional)
    # save_portfolio_snapshot(planned_port_file, signal_date, cash_plan, positions_plan)
    # save_portfolio_snapshot(weekly_port_file,  signal_date, cash_plan, positions_plan)

    # Update a PLANNED live portfolio file (NOT the authoritative one)
    # save_portfolio_snapshot(PLANNED_LIVE_PORT_FILE, signal_date, cash_plan, positions_plan)

    # Append to master trades with de-dup on plan_id
    if not trades_df.empty:
        if master_df.empty:
            master_df = trades_df.copy()
        else:
            if "plan_id" not in master_df.columns:
                master_df["plan_id"] = np.nan

            existing = set(master_df["plan_id"].dropna().astype(str).tolist())
            keep_mask = ~trades_df["plan_id"].astype(str).isin(existing)
            dropped = int((~keep_mask).sum())
            if dropped > 0:
                exec_diag["dropped_deduped_existing"] += dropped

            to_add = trades_df.loc[keep_mask].copy()
            if not to_add.empty:
                master_df = pd.concat([master_df, to_add], ignore_index=True)

    # Append to master rankings (NEW)
    if not rankings_df.empty:
        if master_rankings.empty:
            master_rankings = rankings_df.copy()
        else:
            master_rankings = pd.concat([master_rankings, rankings_df], ignore_index=True)

    exec_diag["orders_generated"] += len(trades_df)
    exec_diag["weeks_written"] += 1

# Final master writes
master_df.to_csv(MASTER_TRADES_FILE, index=False)
master_rankings.to_csv(MASTER_RANKINGS_FILE, index=False)

print("\n=== DONE ===")
print(f"Weeks processed:   {exec_diag['weeks_seen']}")
print(f"Weeks written:     {exec_diag['weeks_written']}")
print(f"Orders generated:  {exec_diag['orders_generated']}")
print("Drops:")
print(f"  missing_price:        {exec_diag['dropped_missing_price']}")
print(f"  min_trade:            {exec_diag['dropped_min_trade']}")
print(f"  cash_floor_buy:       {exec_diag['dropped_cash_floor_buy']}")
print(f"  spy_regime_buy:       {exec_diag['dropped_spy_regime_buy']}")
print(f"  min_new_weight:       {exec_diag['dropped_min_new_weight']}")
print(f"  deduped_existing:     {exec_diag['dropped_deduped_existing']}")
print("\nOutputs:")
print(f"  Master planned trades:        {MASTER_TRADES_FILE}")
print(f"  Master rankings:              {MASTER_RANKINGS_FILE}")
print(f"  Weekly planned trades dir:    {WEEKLY_TRADES_DIR}")
print(f"  Weekly rankings dir:          {WEEKLY_RANKINGS_DIR}")
print(f"  Weekly planned portfolios:    {WEEKLY_PORT_DIR}")
print(f"  Planned live portfolio file:  {PLANNED_LIVE_PORT_FILE}")
print(f"\nNOTE: {LIVE_PORTFOLIO_FILE} was NOT modified (authoritative executions only).")

=== LIVE TRADE GENERATOR (BACKTEST-CONGRUENT, OPTION A) ===
Execution pricing: WED_CLOSE_AS_THU_OPEN_ESTIMATE
Start trading:     2025-12-17
Rebalance day:     Wednesday
Max position cap:  12%
SPY Regime Confirmation Period: 1 day(s)

Loaded universe: 3,591,462 rows
Universe with ATR20 merged: 3,591,462 rows

SPY Regime Statistics:
  Raw regime:       5,107 bull days, 1,935 bear days
  Confirmed regime: 5,107 bull days, 1,935 bear days

Loaded AUTHORITATIVE live portfolio state (from ./27a-2G_live_trading/live_portfolio.csv):
  Cash:      345,000.00
  Positions: 0

Eligible rebalance weeks: 3


=== DONE ===
Weeks processed:   3
Weeks written:     3
Orders generated:  9
Drops:
  missing_price:        0
  min_trade:            0
  cash_floor_buy:       0
  spy_regime_buy:       0
  min_new_weight:       0
  deduped_existing:     0

Outputs:
  Master planned trades:        ./27a-2G_live_trading/master_trades.csv
  Master rankings:              ./27a-2G_live_trading/master_rankings.csv
  We